In [31]:
import pandas as pd
df = pd.read_csv("Data/Labeled_data.csv", sep=',', encoding = "ISO-8859-1")
df.head(10)

,code_index,word,label,tag
0,0,abstractclassmethod,ClASS,C
1,0,ClassPropertyDescriptor,ClASS,C
2,0,classproperty,Method,MTD
3,0,__init__,Method,MTD
4,0,__init__,Method,MTD
5,0,__get__,Method,MTD
6,0,__set__,Method,MTD
7,0,setter,Method,MTD
8,0,ClassPropertyDescriptor,Func_Call,FC
9,0,type,Func_Call,FC


In [33]:
data = df[['code_index','word','tag']]

data.head(20)

,code_index,word,tag
0,0,abstractclassmethod,C
1,0,ClassPropertyDescriptor,C
2,0,classproperty,MTD
3,0,__init__,MTD
4,0,__init__,MTD
5,0,__get__,MTD
6,0,__set__,MTD
7,0,setter,MTD
8,0,ClassPropertyDescriptor,FC
9,0,type,FC


In [34]:
df['tag'].value_counts()

 FC     382
 STR    222
 MTD    155
 Num     97
 LIB     22
 C       14
Name: tag, dtype: int64

In [35]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["word"].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("code_index").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
          
getter = SentenceGetter(data)
sentences = getter.sentences    

In [37]:
print(sentences[1:3])

[[(' json', ' LIB'), (' shutil', ' LIB'), (' BuiltinEntityParser', ' C'), (' _build_builtin_parser', ' MTD'), (' _build_gazetteer_parser', ' MTD'), (' is_builtin_entity', ' MTD'), (' is_gazetteer_entity', ' MTD'), (' is_grammar_entity', ' MTD'), (' find_gazetteer_entity_data_path', ' MTD'), (' _get_gazetteer_entity_configurations', ' MTD'), (' _get_caching_key', ' MTD'), (' dict', ' FC'), (' __init__', ' MTD'), (' _parse', ' MTD'), (' persist', ' MTD'), (' from_path', ' MTD'), (' build', ' MTD'), (' gazetteer_entity_parser', ' STR'), (' sorted', ' FC'), (' FileNotFoundError', ' FC'), (' tuple', ' FC'), (' cls', ' FC'), (' _get_caching_key', ' FC'), (' temp_dir', ' FC'), (' BuiltinEntityParser', ' FC'), (' find_gazetteer_entity_data_path', ' FC'), (' parsers_metadata', ' STR'), (' metadata.json', ' STR'), (' get_all_builtin_entities', ' FC'), (' get_all_gazetteer_entities', ' FC'), (' get_all_grammar_entities', ' FC'), (' parsed_entity', ' FC'), (' ValueError', ' FC'), (' _build_builtin

In [46]:
from math import nan

words = list(set(data["word"].values))
n_words = len(words)
tags = []
for tag in set(data["tag"].values):
    if tag is nan or isinstance(tag, float):
        tags.append('O')
    else:
        tags.append(tag)
n_tags = len(tags)

In [54]:
from future.utils import iteritems

word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {v: k for k, v in iteritems(tag2idx)}
print(tag2idx)

{' LIB': 0, ' C': 1, ' Num': 2, ' STR': 3, ' MTD': 4, ' FC': 5}


In [57]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

maxlen = max([len(s) for s in sentences])

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=n_words - 1)

y = [[tag2idx[t[1]] for t in s] for s in sentences]
y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=n_tags-1)
y = [to_categorical(i, num_classes=n_tags) for i in y]

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [75]:
import tensorflow as tf
from tensorflow import keras


SyntaxError: invalid syntax (<ipython-input-75-d0fd9504c384>, line 3)

In [68]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k
from keras_contrib.layers import CRF

input = Input(shape=(672,))
word_embedding_size = 150

# Embedding Layer
model = Embedding(input_dim=n_words, output_dim=word_embedding_size, input_length=672)(input)

# BI-LSTM Layer
model = Bidirectional(LSTM(units=word_embedding_size, 
                           return_sequences=True, 
                           dropout=0.5, 
                           recurrent_dropout=0.5, 
                           kernel_initializer=k.initializers.he_normal()))(model)
model = LSTM(units=word_embedding_size * 2, 
             return_sequences=True, 
             dropout=0.5, 
             recurrent_dropout=0.5, 
             kernel_initializer=k.initializers.he_normal())(model)

# TimeDistributed Layer
model = TimeDistributed(Dense(n_tags, activation="relu"))(model)  

# CRF Layer
crf = CRF(n_tags)

out = crf(model)  # output
model = Model(input, out)
model.save('C:\Users\user\Desktop\Research-implementation')

NameError: name 'out' is not defined

In [ ]:
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

#Optimiser 
adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

# Compile model
model.compile(optimizer=adam, loss=crf.loss_function, metrics=[crf.accuracy, 'accuracy'])

model.summary()

# Saving the best model only
filepath="ner-bi-lstm-td-model-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Fit the best model
history = model.fit(X, np.array(y), batch_size=256, epochs=10, validation_split=0.2, verbose=1, callbacks=callbacks_list)

# Plot the graph 
plt.style.use('ggplot')

def plot_history(history):
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

plot_history(history)